### MLaSO-d検証用のJupyterNotebook
※MLaSO : Machine Learning assisted Structual Optimization\
PythonとFreeFEMのioを対応させるためにpyfreefemを導入

In [112]:
#fredfemをpythonから呼び出すためのpyfreefemライブラリのインストール
%pip install notebook pyfreefem torch numpy yaml torchsummary
from pyfreefem import FreeFemRunner


  Using cached notebook-7.3.1-py3-none-any.whl.metadata (10 kB)


ERROR: Could not find a version that satisfies the requirement yaml (from versions: none)
ERROR: No matching distribution found for yaml

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


メッシュ生成後、メッシュファイルを読み込んで、radial basis functionに基づくパラメータを定義
$$
    W_{ij}=\exp(-\frac{R_{ij}^2}{r^2})  \tag{1}
$$
$R_{ij}$は各要素における重心座標間の距離を計算した値、$r$は適当な定数

In [ ]:
#メッシュと境界条件を指定して保存
generate_mesh = FreeFemRunner('../src/set-mesh-and-bc.edp')
generate_mesh.import_variables(NN=80.0)
generate_mesh.execute()

In [2]:
#read mesh data and calculate distance between element i and j
import numpy as np

def read_mesh_file(file_path:str):
    # Initialize storage for vertices and triangles
    vertices , triangles = [] ,[]

    try:
        # Open and read the file
        with open(file_path, 'r') as file:
            lines = file.readlines()

        # Parsing logic
        i = 0
        while i < len(lines):
            line = lines[i].strip()

            # Detect 'vertices' section
            if line.lower() == "vertices":
                i += 1  # Move to the line with the number of data points
                num_vertices = int(lines[i].strip())
                i += 1  # Move to the first line of data
                for _ in range(num_vertices):
                    vertex_data = [float(x) for x in lines[i].strip().split()]
                    vertices.append(vertex_data[0:2])
                    print(vertex_data[0:2])
                    i += 1
                continue

            # Detect 'triangles' section
            elif line.lower() == "triangles":
                i += 1  # Move to the line with the number of data points
                num_triangles = int(lines[i].strip())
                i += 1  # Move to the first line of data
                for _ in range(num_triangles):
                    triangle_data = [int(x) for x in lines[i].strip().split()]
                    triangles.append(triangle_data[0:3])
                    print(triangle_data[0:3])
                    i += 1
                continue

            # Move to the next line
            i += 1

        # Convert lists to numpy arrays
        vertices_array = np.array(vertices, dtype=float)
        triangles_array = np.array(triangles, dtype=int)

        # Output the resulting arrays for verification
        return vertices_array, triangles_array-1

    except Exception as e:
        str(e)
        return None

vertices_array , triangles_array = read_mesh_file('../src/mesh/Sh.mesh')
Ne=len(triangles_array)

'''
# freefemでは節点上の密度・感度を計算しているので、今のところ要素間距離を計算する必要はない
# 三角形の頂点座標を取得 (Ne x 3 x 3)
triangle_vertices = vertices_array[triangles_array[:,0:3]]
# 各三角形の重心を計算 (Ne × 3)
centroids = np.mean(triangle_vertices, axis=1)
'''

# Compute pairwise distances using broadcasting
Rij = np.linalg.norm(vertices_array[:, np.newaxis, :] - vertices_array[np.newaxis, :, :], axis=2)
np.save('../src/mesh/Rij',Rij)

# Apply the Gaussian kernel
r=1
wij = np.exp(-Rij**2 / r**2)
np.save('../src/mesh/wij_param',wij)

[2.0, 0.536]
[2.0, 0.542]
[2.0, 0.53]
[1.99565034406, 0.534303866498]
[2.0, 0.548]
[2.0, 0.554]
[1.99565031128, 0.54024895615]
[1.99570399681, 0.546372374703]
[2.0, 0.524]
[1.99135562041, 0.532662577166]
[1.99573741043, 0.528271560909]
[1.98685256363, 0.537175112206]
[1.99117733249, 0.538729794127]
[2.0, 0.56]
[1.99570396402, 0.552315128597]
[1.99128276323, 0.550872206502]
[1.99146588658, 0.544799617301]
[1.99573741043, 0.522325403776]
[2.0, 0.518]
[1.98685256363, 0.531341186148]
[1.99135562041, 0.526771769278]
[1.98219473221, 0.53606766014]
[2.0, 0.571]
[1.98657890101, 0.543279822164]
[1.99128269562, 0.562639813882]
[1.99128269562, 0.556756046048]
[1.98673434768, 0.555501482268]
[1.98657879651, 0.549110396206]
[1.9913555528, 0.520881031054]
[1.99560118662, 0.516293239562]
[2.0, 0.512]
[2.0, 0.582]
[2.0, 0.593]
[1.97746835861, 0.54086368437]
[1.98219458878, 0.530292716045]
[1.98643758156, 0.525236918754]
[1.97746817216, 0.535148486481]
[1.9920129639, 0.579062205953]
[1.98745624208, 0.5

NNの定義\
入力層->隠れ層へのパラメータはradial basis functionの値を基に、隠れ層->出力層へのパラメータは対応する要素間の距離が一定値を下回る場合に与えることとする。\
感度は節点上で計算されているので、うまく置き換える必要がある。

In [126]:
#NNの定義
import torch
import torch.nn as nn
from torchsummary import summary

class sensitivity_prediction(nn.Module):
    def __init__(self, Ne, Rmin, E_min):
        super(sensitivity_prediction, self).__init__()
        
        # 式に基づく全結合層（入力層から隠れ層）の重み行列を定義（バイアスなし）
        self.fc1_weight = nn.Parameter(torch.empty(Ne,Ne))
        
        # 局所的な接続ルール
        self.connection_rule = torch.tensor(np.load('../src/mesh/Rij.npy') - Rmin,dtype=torch.float32)  # 接続ルール
        self.sparse_indices = torch.argwhere(self.connection_rule < 0).clone().detach()  # 負の値の位置だけ保持
        self.sparse_weights = nn.Parameter(torch.rand(self.sparse_indices.size(0),dtype=torch.float32)*E_min)  # 接続される重みだけを定義

    def initialize_fc1_weights(self):
        # fc1の重み行列をカスタム式で初期化
        self.wij=np.load('../src/mesh/wij_param.npy')
        with torch.no_grad():
            self.fc1_weight.data=torch.tensor(self.wij,dtype=torch.float32)

    def forward(self, x):
        print("forward")
        # x の形状を確認
        if x.ndim == 1:
            x = x.unsqueeze(0)  # バッチ次元を追加

        if torch.isnan(x).any() or torch.isinf(x).any():
            raise ValueError("x contains NaN or Inf")
        if torch.isnan(self.fc1_weight).any() or torch.isinf(self.fc1_weight).any():
            raise ValueError("fc1_weight contains NaN or Inf")
        
        # 入力層から隠れ層への全結合演算
        try:
            x = x @ self.fc1_weight.T
        except Exception as e:
            print("Error during matrix multiplication")
            raise e

        # スパース接続による隠れ層から出力層への演算
        i_indices = self.sparse_indices[:, 0]  # 隠れ層のインデックス
        j_indices = self.sparse_indices[:, 1]  # 出力層のインデックス

        # 必要な要素だけを抽出し重みと掛け算
        #sparse_connections = x.index_select(1, j_indices) * self.sparse_weights.view(1, -1)

        # 出力テンソルを初期化し、scatter_add_で値を集計
        #output = torch.zeros(x.size(0), self.fc1_weight.size(0), device=x.device)
        #output.index_add_(1, i_indices, sparse_connections)
        
        # スパース行列を構築 (Ne, Ne)
        Ne = self.fc1_weight.size(0)
        indices = torch.stack([i_indices, j_indices])   # shape: [2, #connections]
        values = self.sparse_weights                    # shape: [#connections]
        sparse_mat = torch.sparse_coo_tensor(indices, values, (Ne, Ne), device=x.device)
        dense_mat = sparse_mat.to_dense()
        # スパース行列での計算 (batch_size, Ne) @ (Ne, Ne) -> (batch_size, Ne)
        # torch.sparse.mmを用いてスパース行列との行列積を計算
        output = x @ dense_mat

        return output

#model = sensitivity_prediction(5883,0.01,0.01)
#summary(model,(1,5883))

#### Algorithm3：On-the-job training
変数の数が多いためクラスを使わないと定義が大変\
#3はコードの実行中である限りパラメータが保存されるので考える必要がない。#22も同様。\
k=0のときにdeltaをどのように決めておいたらいいのかわからない、というかそもそも訓練しないのでは？
|変数名|説明|サイズ|Globalでの記録|コード内変数|データ型|
| :--- | :--- | :--- | :--- | :--- | :--- |
|$d$|応力解析から得られた感度|1 x Ne|要|d|torch.tensor|
|$s^{k_r}$|正規化のためのパラメータ|2 x 2|要|s|torch.tensor|
|$\tilde{d}^{k}$|$k$における感度のリスケール値|1 x Ne|不要|d_tilde_k|torch.tensor|
|$\tilde{d}^{k_r}$|$k_r$における感度のリスケール値|1 x Ne|不要|d_tilde_kr|torch.tensor|
|$er^{k_r}$|予測誤差の蓄積量|1 x Ne|要|er|torch.tensor|
|$\epsilon^{k_r}$|予測誤差|1 x Ne|要|epsilon|torch.tensor|
|$\beta$|Dynamic Smoothing|1 x Ne|不要|beta|torch.tensor|
|$\delta^{k-1}$|訓練における入力ベクトル|1 x Ne|不要|delta|torch.tensor|
|$\bar{d}^{k}$|$k$における感度の保管|1 x Ne|要|d_bar_k|torch.tensor|
|$\bar{d}^{k_r}$|$k_r$における感度の保管|1 x Ne|要|d_bar_kr|torch.tensor|
|$z_{k_n}$|予測と真値の誤差||||
|$\tau_{t}$|誤差の収束条件||||

In [ ]:
#--------------------------------------------------
# Implement On-the-job training
#--------------------------------------------------
# Run FEA with saved mesh data
import torch.optim as optim
import numpy as np

def on_the_job_training(model,gamma:float,beta_min:float,tau_t:float,max_epoch:int,Ne:int=5883,ks:int=5)->float:
    global k,kr,comp     #iteration index
    global rho,d,s,er,epsilon,d_bar_k,d_bar_kr
    
    run_fem = FreeFemRunner('../src/2Delastic-FEA.edp')
    rho_np=np.array(rho.detach().tolist()).ravel()
    print(rho_np)
    run_fem.import_variables(E=210e9,nu=0.21,Volcoef=0.5,lamG=0.1,SigG=0.03,rhomin=0.001,rhomax=1.0,regR=0.04,kr=float(kr),rho=rho_np)   #1
    result = run_fem.execute()
    
    d = torch.tensor(result["d"])  #2
    comp.append(result["comp"])
    
    s[1] = s[0].clone()        
    s[0] = torch.tensor([torch.max(d),torch.min(d)])   #4

    #temporal variables
    d_tilde_k = (d-s[0,1])/(s[0,0]-s[0,1])
    d_tilde_kr= (d-s[0,1])/(s[0,0]-s[0,1])

    if k>0:                                                         #6
        delta=d_bar_k.clone()                                       #7
        
        if kr>=2:                                                   #9
            er=torch.max(torch.ones([Ne]), er.clone()+gamma*(epsilon-torch.min(epsilon))/(torch.max(epsilon)-torch.min(epsilon))) #10
            
        beta=torch.max(torch.ones([Ne])*beta_min,torch.min(torch.ones([Ne]),1/er))                    #13
        beta = torch.clamp(beta, min=1e-6, max=1-1e-6)
        
        d_bar = d_tilde_k * beta + d_bar_kr*(1-beta)
        d_bar_k , d_bar_kr = d_bar.clone() , d_bar.clone()
        
        pred_error = model(delta)-d_bar_k
        epsilon = torch.abs(pred_error)                             #15
    else:                                                           #16
        d_bar_k, d_bar_kr = d_tilde_k.clone(), d_tilde_kr.clone()                   #17

    model.train()
    loss_fn = nn.MSELoss()
    optimizer = optim.SGD(
        model.parameters(),lr=1e-3,weight_decay=0                   #パラメータは適当
    )

    if k>=ks:                              #論文には記載されていないが、感度の変化が大きい最初の数ステップは切り捨てる
        for _ in range(max_epoch):         #19
            optimizer.zero_grad()
            pred=model(delta)                   #20
            pred = torch.clamp(pred, min=1e-6, max=1e6)
            print(f"Pred Min: {pred.min()}, Max: {pred.max()}, Mean: {pred.mean()}")
            print(f"d_bar_k Min: {d_bar_k.min()}, Max: {d_bar_k.max()}, Mean: {d_bar_k.mean()}") 
            loss = loss_fn(pred,d_bar_k)
            print("Loss before backward:", loss.item())
            
            loss.backward(retain_graph=True)
            optimizer.step()
            
            if float(loss) <= tau_t:               #21,22
                break
                             
    
    d = s[0,1]+(d_bar_k-torch.min(d_bar_k))/(torch.max(d_bar_k)-torch.min(d_bar_k))*(s[0,0]-s[0,1])
    print('d',d)
    kr+=1
        


#### Algorithm2：On-the-job prediction
|変数名|説明|サイズ|Globalでの記録|コード内変数|データ型|
| :--- | :--- | :--- | :--- | :--- | :--- |
|$d$|応力解析から得られた感度|1 x Ne|要|d|torch.tensor|
|$\hat{s}^{k_r}$|正規化のためのパラメータ|2 x 2|要|s_hat|torch.tensor|
|$\bar{d}^{k}$|$k$における感度の保管|1 x Ne|要|d_bar_k|torch.tensor|

モデルの出力をd_bar_kに代入するが、すべて0になってしまう。

In [97]:
#On-the-job predictionの実行
def on_the_job_prediction(model):
    global d_bar_k,d,kp
    ws=(s[0,0]-s[0,1])/(s[1,0]-s[1,1])
    bs=s[0,0]
    s_hat=ws*s[0]+bs
    model.eval()
    d_bar_k = model(d_bar_k)
    print(f'pred->d_bar_k:{kp}',d_bar_k)
    d = s_hat[1]+(d_bar_k-torch.min(d_bar_k))/(torch.max(d_bar_k)-torch.min(d_bar_k))*(s_hat[0]-s_hat[1])
    kp+=1

#### Algorithm1：MLaSO-d
1. 初期化
2. 反復計算の開始

In [127]:
#MLaSO-dの実行
import yaml
import numpy as np
import os

torch.autograd.set_detect_anomaly(True)
print(os.getcwd())

with open('../config/MLaSO-d_param.yaml','r') as yml:
    config = yaml.safe_load(yml)
  
beta_min = config['MLaSO_d']['beta_min']
r_l      = config['MLaSO_d']['r_l']
tau_t    = config['MLaSO_d']['tau_t']
ks       = config['MLaSO_d']['ks']
kr       = config['MLaSO_d']['kr']
kp       = config['MLaSO_d']['kp']
max_loop = config['MLaSO_d']['max_loop']
Ne       = config['MLaSO_d']['Ne']
r_min    = config['MLaSO_d']['r_min']

max_epoch= config['on_the_job_training']['max_epoch']
gamma    = config['on_the_job_training']['gamma_0']
delta_gamma = config['on_the_job_training']['delta_gamma']
R_min    = config['on_the_job_training']['R_min']   
E_min    = config['on_the_job_training']['E_min']

rho=torch.ones([Ne],dtype=torch.float32)
d=torch.zeros([Ne],dtype=torch.float32)
s=torch.zeros([2,2],dtype=torch.float32)
er=torch.zeros([Ne],dtype=torch.float32)
epsilon=torch.zeros([Ne],dtype=torch.float32)
d_bar_k=torch.ones([Ne],dtype=torch.float32)
d_bar_kr=torch.ones([Ne],dtype=torch.float32)
comp=[]

model=sensitivity_prediction(Ne,R_min,E_min)

dt=0.2
diff=0.01

for k in range(max_loop):
    if k<=ks or np.mod(np.max([k-ks,1]),2)==0:
        on_the_job_training(model,gamma,beta_min,tau_t,max_epoch)
        print(f'on-the-job-training : {kr}')
    else:
        on_the_job_prediction(model)
        print(f'on-the-job-prediction : {kp}')
        
    rho = rho + dt * d
        
    ##sensitivity filter
    #dist = torch.tensor(np.load('../src/mesh/Rij.npy'))
    #weight = torch.clamp(r_min-dist, min=0)
    #weighted_d = torch.matmul(weight, d)
    #weighted_rho= torch.matmul(torch.clamp(rho, min=1e-3), weight)
    ## 分子と分母を集計してノードごとの更新を計算
    #sum_Numerator = torch.sum(weighted_d, dim=1)
    #sum_Denominator = torch.sum(weighted_rho, dim=1)
    ## 感度を更新
    #d = sum_Numerator / sum_Denominator
    #
    #B=-d/lambda     #ラグランジュ乗数
    ##optimality criteria method
    #for i in range(len(rho)):
    #    if rho[i]*

    #update dynamic weightage
    if kr>=4:
        diff_obj_kr   = abs((comp[kr-1]-comp[kr-2])/abs(comp[kr-1]-comp[kr-2])+(comp[kr-2]-comp[kr-3])/abs(comp[kr-2]-comp[kr-3]))
        diff_obj_kr_1 = abs((comp[kr-1]-comp[kr-2])/abs(comp[kr-1]-comp[kr-2])-(comp[kr-2]-comp[kr-3])/abs(comp[kr-2]-comp[kr-3]))
        if diff_obj_kr<=diff and diff_obj_kr_1<=diff:
            gamma=gamma+delta_gamma

c:\Users\akkii\Documents\UT\MiD_Lab\code\mid_lab-akimoto\pj-TO_Speedup\notebooks
[1. 1. 1. ... 1. 1. 1.]
d tensor([6.8076, 7.9491, 6.0647,  ..., 1.1570, 1.4618, 1.3044])
on-the-job-training : 1
[2.36151409 2.58981657 2.21293831 ... 1.23140001 1.29236245 1.2608732 ]
forward
d tensor([ 2.9280,  3.0345,  2.6277,  ..., -0.0473, -0.0142, -0.0323])
on-the-job-training : 2
[2.94712043 3.19672346 2.73847604 ... 1.22194183 1.28952122 1.25442052]
forward
d tensor([[ 4.4093,  4.9893,  3.8995,  ..., -0.0759, -0.0691, -0.0739]],
       grad_fn=<AddBackward0>)
on-the-job-training : 3
[3.82897258 4.19457674 3.51837325 ... 1.20676219 1.27569509 1.23963773]
forward
d tensor([[ 4.2564,  4.7687,  3.7834,  ..., -0.0333, -0.0564, -0.0451]],
       grad_fn=<AddBackward0>)
on-the-job-training : 4
[4.68025064 5.14831495 4.2750535  ... 1.2001121  1.2644192  1.23062241]
forward
d tensor([[4.0970, 4.5686, 3.6321,  ..., 0.0715, 0.0376, 0.0549]],
       grad_fn=<AddBackward0>)
on-the-job-training : 5
[5.49965048 6

c:\Users\akkii\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Error detected in MmBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\akkii\AppData\Roaming\Python\Python311\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\akkii\AppData\Roaming\Python\Python311\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\akkii\AppData\Roaming\Python\Python311\site-packages\ipykernel\kernelapp.py", line 739, in start
    self.io_loop.start()
  File "C:\Users\akkii\AppData\Roaming\Python\Python311\site-packages\tornado\platform\asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "c:\Users\akkii\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [5883, 5883]], which is output 0 of AsStridedBackward0, is at version 1; expected version 0 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!